<a href="https://colab.research.google.com/github/gabriellaaileen/EDA/blob/main/Sentiment_Analysis_LGBT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Libraries**

In [ ]:
!pip install textblob
!pip install tweepy==4.10.0

In [ ]:
!pip install pycountry
!pip install langdetect

In [ ]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
import warnings; warnings.simplefilter('ignore')
import nltk

try:
    import google.colab
    IN_COLAB = True
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataDDGsna.py
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/contoh.pdf
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_id.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_en.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/kata_dasar.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-ind-def.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-msa-all.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/ind_SA.csv
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger
    !wget -P data/ https://raw.githubusercontent.com/gabriellaaileen/EDA/main/kamus_slang.csv
    !wget -P data/ https://raw.githubusercontent.com/gabriellaaileen/EDA/main/kata_positif.txt
    !wget -P data/ https://raw.githubusercontent.com/gabriellaaileen/EDA/main/kata_negatif.txt

    !pip install --upgrade spacy python-crfsuite unidecode textblob sastrawi sklearn-pycrfsuite
    !pip install --upgrade unidecode twython tweepy beautifulsoup4 tika
    !python -m spacy download en
    !python -m spacy download xx
    !python -m spacy download en_core_web_sm
    nltk.download('popular')
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

In [6]:
import taudataDDGsna as tau
import tweepy, json, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
"Done"

'Done'

In [ ]:
!pip install googletrans==3.1.0a0


In [8]:
from googletrans import Translator
translator = Translator()

##**Crawling & Saving Data**

In [33]:
consumerKey = #Input
consumerSecret = #Input
accessToken = #Input
accessTokenSecret = #Input
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit=True, timeout=180, retry_count=5, retry_delay=3)
usr_ = api.verify_credentials()

In [72]:
qry = '"LGBT" OR "LGBTQ" OR "LGBTQ+" OR "pride month" OR "lesbian" OR "lesbi" OR "transgender" OR "homo" OR "gay" OR "homoseksual" OR "biseks" OR "biseksual" OR "bisex" since:2022-03-01'
N = 5000 # jumlah N tweet "terkini" yang ingin diambil
T = []
for post in tqdm(tweepy.Cursor(api.search_tweets, q=qry, lang='id', tweet_mode='extended').items(N)):
    T.append(post)
    
"Done"

1711it [00:40, 41.98it/s]


'Done'

In [73]:
# T bukan sekedar list of tweets 
tweet = [t._json for t in T]
print(' Berhasil mendapatkan {} tweets'.format(len(tweet)))

 Berhasil mendapatkan 1711 tweets


In [74]:
# Datanya berbentuk JSON
tweet[0]['created_at'], tweet[0]['full_text']

('Thu Jun 09 04:48:54 +0000 2022', '@convomf Knpa LGBT🗿')

In [75]:
# Contoh mengakses data spesifik pada tweet yang pertama:
print('tweet pertama oleh "{}" : "{}"'.format(tweet[0]['user']['screen_name'],tweet[0]['full_text']))

tweet pertama oleh "putcatdoang" : "@convomf Knpa LGBT🗿"


In [76]:
# Contoh mengambil hanya data tweet
D = [t['full_text'] for t in tweet]
D[:10] # 5 tweet pertama

['@convomf Knpa LGBT🗿',
 '@maheenm26727540 Koi gsl nahi dobra repair kar lain gay',
 'Beredar video empat orang laki-laki sedang berpangkuan dengan pasangannya masing-masing.\xa0\n\nhttps://t.co/5naB2G18Iy\n\n#republika #lgbt #pancoran #jakarta https://t.co/Im0KTFMuYv',
 '"mbak minta tolong pegang perut saya ya biar debay nya cantik kayak embaknya" amiinn tapi masalahnya saya suka nonton homo ibuu semoga gak nurun yaa😭😭😭😭',
 'Lah kan lagu tuh masuknya seni, gue bukan pro LGBT tapi menyukai seni bukan LGBT nya https://t.co/uRzzGoePjk',
 'lgbt https://t.co/Y2K3nHUu7U',
 'RT @Meilansyahriza: Sudah menjadi Rahasia umum.... Siapa yang ada di belakang LGBT.... Sama mereka yang ada di belakang VAKSINISASI... Dan…',
 'Ego is wlw',
 '"Homo-Lehrsaal"',
 'Lagi dengerin lagu LGBT katanya hehe']

In [77]:
import json 
def saveTweets(tweets, file='Tweets.json'): #in Json Format
    with open(file, 'w') as f:
        for t in tweets:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass

In [78]:
# Menyimpan hasil crawling twitter
fileName = 'Tweets.json'
saveTweets(tweet, file=fileName)
print('Saved to '+fileName)

Saved to Tweets.json


In [79]:
import pandas as pd
df = pd.read_json('/content/Tweets.json', lines=True)
df.to_csv ('/content/Tweets.csv', index = None)

In [80]:
df.head(2)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,retweeted_status
0,2022-06-09 04:48:54+00:00,1534759419417751552,1534759419417751552,@convomf Knpa LGBT🗿,False,"[9, 19]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.534669e+18,...,0,False,False,in,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-09 04:48:33+00:00,1534759330196627456,1534759330196627456,@maheenm26727540 Koi gsl nahi dobra repair kar...,False,"[17, 55]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.534584e+18,...,0,False,False,in,NaN,NaN,NaN,NaN,NaN,NaN


##**Cleaning Data**

In [81]:
#We use only the full_text data
tweet_list=pd.DataFrame(df['full_text'])
tweet_list.head()

,full_text
0,@convomf Knpa LGBT🗿
1,@maheenm26727540 Koi gsl nahi dobra repair kar...
2,Beredar video empat orang laki-laki sedang ber...
3,"""mbak minta tolong pegang perut saya ya biar d..."
4,"Lah kan lagu tuh masuknya seni, gue bukan pro ..."


In [82]:
#drop duplicate tweets
#tweet_list.drop_duplicates(inplace = True)

In [83]:
#This function will take in a few patterns to look for, such as links, @usernames, hashtags, numbers etc. 
#and replace these patterns with an empty string.

def clean_text(text):  
    pat1 = r'@[^ ]+'                   
    pat2 = r'https?://[A-Za-z0-9./]+'  
    pat3 = r'\'s'                      
    pat4 = r'\#\w+'                     
    pat5 = r'&amp '                     
    pat6 = r'[^A-Za-z\s]'               
    combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))
    text = re.sub(combined_pat,"",text).lower()
    return text.strip()

In [84]:
tweet_list["cleaned_tweet"] = tweet_list["full_text"].apply(clean_text)

In [85]:
#However we may have some empty tweets as we removed quite a lot of content, so lets drop these.
tweet_list = tweet_list [ ~(tweet_list["cleaned_tweet"] =="")]

In [86]:
#Replace slang using dictionary from kamus_slang.csv
d = dict()
f = open("data/kamus_slang.csv")
for line in f:
    line = line.strip('\n')
    (key, val) = line.split(",")
    d[key] = val

# Replace slang using dictionary from slang.txt 
df=open('data/slang.txt',"r",encoding="utf-8", errors='replace')
slangS = df.readlines(); df.close()
# pisahkan berdasarkan ':'
slangS = [t.split(":") for t in slangS]
slangS = [[k.strip(), v.strip()] for k,v in slangS]
slangS = {k:v for k,v in slangS}

In [87]:
for j in range (len(tweet_list)):
    tweet = tweet_list.iloc[j]['cleaned_tweet']
    tweet = TextBlob(tweet).words
    for i,t in enumerate(tweet):
      if t in d.keys():
        tweet[i] = d[t]
      elif t in slangS.keys():
        tweet[i] = slangS[t]
    tweet_list.iloc[j]['cleaned_tweet']=' '.join(tweet)

In [88]:
tweet_list.head()

,full_text,cleaned_tweet
0,@convomf Knpa LGBT🗿,kenapa lgbt
1,@maheenm26727540 Koi gsl nahi dobra repair kar...,koi gsl nahi dobra repair kar lain gay
2,Beredar video empat orang laki-laki sedang ber...,beredar video empat orang lakilaki sedang berp...
3,"""mbak minta tolong pegang perut saya ya biar d...",mbak minta tolong pegang perut saya iya biar d...
4,"Lah kan lagu tuh masuknya seni, gue bukan pro ...",lah akan lagu itu masuknya seni saya bukan pro...


In [89]:
#Terdapat beberapa tweet yang ditulis dalam bahasa lain dan diduga tidak ditulis oleh WNI
#Dilakukan deteksi bahasa yang dipakai:
language=[]
for i in range (len(tweet_list)):
  tweet = tweet_list.iloc[i]['cleaned_tweet']
  detected_language=translator.detect(tweet).lang
  if isinstance(detected_language, list)==True:
    detected_language=tuple(detected_language)
  language.append(detected_language)
tweet_list['language']=language

In [90]:
#Tweet dengan bahasa selain id, en, (id,ms), (ms,id), dan (jw,id) akan dihapus dari dataframe 
tweet_list.language.value_counts()

id          1117
en           179
ms           111
(id, ms)      64
hi            55
(ms, id)      41
es            21
jw            18
ar            13
su            11
ja             7
tl             6
gu             5
sw             5
tr             3
rw             3
bn             2
mi             2
it             2
ha             2
so             2
te             2
(jw, id)       2
de             2
zh-CN          2
(ms, su)       2
ml             2
ig             2
zu             2
(mi, so)       1
ca             1
qu             1
ro             1
cs             1
sk             1
fy             1
et             1
(pt, es)       1
(id, su)       1
(ms, en)       1
pt             1
bg             1
haw            1
(tl, en)       1
(ja, ha)       1
(gu, hi)       1
da             1
(la, id)       1
uz             1
(id, tr)       1
la             1
(su, jw)       1
bm             1
(hi, bn)       1
el             1
Name: language, dtype: int64

In [91]:
lang=['id', 'en', ('id','ms'), ('ms','id'), ('jw','id')]
tweet_list = tweet_list[tweet_list['language'].isin(lang)]

In [92]:
tweet_list.head(10)

,full_text,cleaned_tweet,language
0,@convomf Knpa LGBT🗿,kenapa lgbt,"(ms, id)"
2,Beredar video empat orang laki-laki sedang ber...,beredar video empat orang lakilaki sedang berp...,id
3,"""mbak minta tolong pegang perut saya ya biar d...",mbak minta tolong pegang perut saya iya biar d...,id
4,"Lah kan lagu tuh masuknya seni, gue bukan pro ...",lah akan lagu itu masuknya seni saya bukan pro...,id
5,lgbt https://t.co/Y2K3nHUu7U,lgbt,en
6,RT @Meilansyahriza: Sudah menjadi Rahasia umum...,rt sudah menjadi rahasia umum siapa yang ada d...,id
8,"""Homo-Lehrsaal""",homolehrsaal,en
9,Lagi dengerin lagu LGBT katanya hehe,lagi dengarkan lagu lgbt katanya he,"(ms, id)"
10,@TretanMuslim Mampus kau homo,mampus kamu homo,id
11,RT @rendemour: TAEKOOK AU\n\n— KAZE 風 —\n\nSeo...,rt taekook angkatan udara kaze seorang ambisiu...,id


**Stopwords sengaja tidak dihilangkan agar tidak mengubah makna saat melakukan sentiment analysis menggunakan Google Translate dan TextBlob**

##**Sentiment Analysis**

<p>Sentiment menghasilkan Tuple berpasangan (<strong>Polaritas</strong>, <strong>Subjectivitas</strong>).&nbsp;</p>

<p>Polaritas memiliki nilai [-1, 1] ==&gt; negative~positive Sentimen</p>

<p>Subjectivity memiliki nilai antara 0 sampai 1, dimana 0 paling objective dan 1 paling subjective.</p>

##**Naive Bayes Classifier**

In [114]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
 
def word_feats(words):
    return dict([(word, True) for word in words])

def bentukClassifier(wPos, wNeg): # ,Nt
    positive_features = [(word_feats(pos), 'pos') for pos in wPos]
    negative_features = [(word_feats(neg), 'neg') for neg in wNeg]
    #neutral_features = [(word_feats(neu), 'neu') for neu in Nt]
    train_set = negative_features + positive_features# + neutral_features
    return NaiveBayesClassifier.train(train_set)

def prediksiSentiment(kalimat, wPos, wNeg, negasi):
    pos, neg = 0.0, 0.0
    posWords, negWords = [], []
    K = tau.cleanText(kalimat)
    for w in wPos:
        if w in K:
            for ww in negasi:
                kebalikan = False
                inverted = ww+' '+w
                if inverted in K:
                    negWords.append(inverted)
                    kebalikan = True
                    break
            if not kebalikan:
                posWords.append(w)
    for w in wNeg:
        if w in K:
            for ww in negasi:
                kebalikan = False
                inverted = ww+' '+w
                if inverted in K:
                    posWords.append(inverted)
                    kebalikan = True
                    break
            if not kebalikan:
                negWords.append(w)
    
    nPos, nNeg = len(posWords), len(negWords)
    sum_ = nPos + nNeg
    if sum_ == 0 or nPos==nNeg:
        return 'netral', 0.0
    else:
        nPos, nNeg = nPos/sum_, nNeg/sum_
        if nPos>nNeg and nPos>0.01:
            return 'positif', nPos
        elif nNeg>nPos and nNeg<-0.01:
            return 'negatif', nNeg
        else:
            return 'netral', (nPos + nNeg)/2

In [115]:
#Buat daftar kata positif dalam Bahasa Indonesia
df=open('/content/data/kata_positif.txt',"r",encoding="utf-8", errors='replace')
wPos = df.readlines(); df.close()
wPos = [k.strip() for k in wPos]
wPos = tuple(k for k in wPos)

In [116]:
#Buat daftar kata negatif dalam Bahasa Indonesia
df=open('/content/data/kata_negatif.txt',"r",encoding="utf-8", errors='replace')
wNeg = df.readlines(); df.close()
wNeg = [k.strip() for k in wPos]
wNeg = tuple(k for k in wPos)

In [117]:
wordS = (wPos, wNeg)
negasi = ['ga', 'tidak','gak','g','tdk','tidk','tdak','anti']

In [118]:
sentimen=[]
score=[]
for i in range (len(tweet_list)):
  sentence = tweet_list.iloc[i]['cleaned_tweet']
  (sen, val)=prediksiSentiment(sentence, wPos, wNeg, negasi)
  sentimen.append(sen)
  score.append(val)
tweet_list['sentimen_NBC']=sentimen
tweet_list['score_NBC']=score

In [119]:
tweet_list[["cleaned_tweet", "sentimen_NBC","score_NBC"]].head(10)

,cleaned_tweet,sentimen_NBC,score_NBC
0,lgbt,netral,0.0
2,beredar video orang lakilaki berpangkuan pasan...,netral,0.0
3,mbak tolong pegang perut iya biar debay nya ca...,netral,0.0
4,lagu masuknya seni profesional lgbt menyukai s...,netral,0.0
5,lgbt,netral,0.0
6,rahasia lgbt vaksinisasi,netral,0.0
8,homolehrsaal,netral,0.0
9,dengarkan lagu lgbt,netral,0.0
10,mampus homo,netral,0.0
11,taekook angkatan udara kaze ambisius dhia noah...,netral,0.0


In [120]:
tweet_list['sentimen_NBC'].value_counts()

netral    1403
Name: sentimen_NBC, dtype: int64

Hasil analisis sentimen menggunakan algoritma Naive Bayes dengan daftar kata positif dan negatif kurang memberikan hasil yang mencerminkan sentimen. Hal ini terlihat dari hasil sentimen yang seluruhnya 'netral'. Kasus ini kemungkinan disebabkan daftar kata positif dan negatif yang digunakan terbatas, sehingga mempengaruhi scoring.

##**Google Translate x TextBlob**

In [93]:
def SenSubModMood_ID(kalimat):
    K = translator.translate(kalimat)
    K = TextBlob(K.text)
    pol,sub = K.sentiment
    if pol>0:
        pol='positive'
    elif pol<0:
        pol='negative'
    else:
        pol = 'netral'
    if sub>0.5:
        sub = 'subjektif'
    else:
        sub = 'objektif'
    return pol, sub

In [94]:
polarity=[]
subjectivity=[]
print("Running sentiment process")
for row in tweet_list.itertuples():
    tweet = tweet_list.at[row[0], 'cleaned_tweet']
    pol, sub=SenSubModMood_ID(tweet)
    polarity.append(pol)
    subjectivity.append(sub)
tweet_list['sentiment']=polarity
tweet_list['subjectivity']=subjectivity

Running sentiment process


In [95]:
tweet_list[["cleaned_tweet", "sentiment","subjectivity"]].head(10)

,cleaned_tweet,sentiment,subjectivity
0,kenapa lgbt,netral,objektif
2,beredar video empat orang lakilaki sedang berp...,netral,objektif
3,mbak minta tolong pegang perut saya iya biar d...,positive,subjektif
4,lah akan lagu itu masuknya seni saya bukan pro...,positive,objektif
5,lgbt,netral,objektif
6,rt sudah menjadi rahasia umum siapa yang ada d...,negative,subjektif
8,homolehrsaal,netral,objektif
9,lagi dengarkan lagu lgbt katanya he,netral,objektif
10,mampus kamu homo,positive,subjektif
11,rt taekook angkatan udara kaze seorang ambisiu...,positive,subjektif


In [96]:
df=pd.crosstab(index=tweet_list['sentiment'],columns=tweet_list['subjectivity'])
df

subjectivity,objektif,subjektif
sentiment,,
negative,128,130
netral,514,29
positive,210,392


In [113]:
len(tweet_list)

1403

##**Visualisasi**

In [97]:
# Loading Stopwords: Ada beberapa cara
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()

NLTK_StopWords = stopwords.words('english')
Sastrawi_StopWords_id = factory.get_stop_words()

df=open('data/stopwords_en.txt',"r",encoding="utf-8", errors='replace')
en_stop = df.readlines()
df.close()
en_stop = [t.strip().lower() for t in en_stop]

df=open('data/stopwords_id.txt',"r",encoding="utf-8", errors='replace')
id_stop = df.readlines()
df.close()
id_stop = [t.strip().lower() for t in id_stop]

N = 10
print(NLTK_StopWords[:N])
print(Sastrawi_StopWords_id[:N])
print(en_stop[:N])
print(id_stop[:N])
print(len(Sastrawi_StopWords_id), len(id_stop), len(NLTK_StopWords), len(en_stop))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua']
['&gt', '&lt', '&nbsp', 'a', 'able', 'about', 'above', 'abst', 'accordance', 'according']
['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir']
126 758 179 2659


In [98]:
# Tips: selalu rubah list stopwords ke bentuk set, karena di Python jauh lebih cepat untuk cek existence di set ketimbang list
NLTK_StopWords = set(NLTK_StopWords)
Sastrawi_StopWords_id = set(Sastrawi_StopWords_id)
en_stop = set(en_stop)
id_stop = set(id_stop)

In [99]:
# Cara menggunakan stopwords
from textblob import TextBlob

for i in range(len(tweet_list)):
  tweet=tweet_list.iloc[i]['cleaned_tweet']
  Tokens = TextBlob(tweet).words # Tokenisasi 
  T2 = [t for t in Tokens if t not in id_stop] # Sastrawi_StopWords_id Personal_StopWords_en Personal_StopWords_id
  T2 = [t for t in T2 if t not in en_stop] # Sastrawi_StopWords_id Personal_StopWords_en Personal_StopWords_id
  tweet_list.iloc[i]['cleaned_tweet']=(' '.join(T2))
# Catatan: Selalu lakukan Stopword filtering setelah tokenisasi (dan normalisasi).

In [100]:
stemmed=[]
# Lemmatizer dengan Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stemmer = StemmerFactory().create_stemmer()

for i in range (len(tweet_list)):
  tweet = tweet_list.iloc[i]['cleaned_tweet']
  stemmed.append(stemmer.stem(tweet))

tweet_list['stemmed_tweet']=stemmed

# Ada beberapa hal yang berbeda antara Sastrawi dan modul-modul diatas.
# Apa sajakah?

In [101]:
tweet_list[['cleaned_tweet','stemmed_tweet']].head(10)

,cleaned_tweet,stemmed_tweet
0,lgbt,lgbt
2,beredar video orang lakilaki berpangkuan pasan...,edar video orang lakilaki pangku pasang masing...
3,mbak tolong pegang perut iya biar debay nya ca...,mbak tolong pegang perut iya biar debay nya ca...
4,lagu masuknya seni profesional lgbt menyukai s...,lagu masuk seni profesional lgbt suka seni lgb...
5,lgbt,lgbt
6,rahasia lgbt vaksinisasi,rahasia lgbt vaksin
8,homolehrsaal,homolehrsaal
9,dengarkan lagu lgbt,dengar lagu lgbt
10,mampus homo,mampus homo
11,taekook angkatan udara kaze ambisius dhia noah...,taekook angkat udara kaze ambisius dhia noah r...


In [102]:
tweet_list.to_csv('Preprocessed_Tweets.csv')

In [111]:
#Hasil tokenisasi dan stemming yang akan digunakan untuk visualisasi
df=pd.DataFrame(tweet_list['stemmed_tweet'])
df.to_csv(r'visualisasi_overall.txt', header=None, index=None, sep='\t', mode='a')

In [112]:
sen=['positive','negative','netral']
for sentimen in sen:
  df=tweet_list.loc[tweet_list['sentiment']==sentimen]
  df.to_csv("visualisasi_"+sentimen+".txt", header=None, index=None, sep='\t', mode='a')